In [45]:
import json

# Load secrets


In [46]:
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

# Check out repo


In [47]:
!rm -r any2json
!git clone --filter=blob:none  https://github.com/btseytlin/any2json.git
!cd any2json && git fetch && git reset --hard origin/main

Cloning into 'any2json'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 449 (delta 59), reused 57 (delta 29), pack-reused 362 (from 1)
Receiving objects: 100% (449/449), 42.12 KiB | 1.62 MiB/s, done.
Resolving deltas: 100% (240/240), done.
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 49 (delta 1), reused 3 (delta 0), pack-reused 15 (from 1)
Receiving objects: 100% (49/49), 250.41 KiB | 2.32 MiB/s, done.
Resolving deltas: 100% (1/1), done.
HEAD is now at f69887c Vibin


In [48]:
!cd any2json && uv -q pip install -e . && uv -q pip install vllm --torch-backend=auto

# Download dataset

In [49]:
from datasets import load_dataset

DATASET_ID = 'btseytlin/any2json'

dataset = load_dataset(DATASET_ID)

print(len(dataset['train']))
print(len(dataset['test']))
dataset['train']

110676
77946


Dataset({
    features: ['input_data', 'schema', 'output', 'meta'],
    num_rows: 110676
})

# Inspect dataset

In [50]:
dataset['train'][0]['meta']

{'group': 516,
 'input_chunk_content_type': 'STRING',
 'input_chunk_id': 32070,
 'is_test': False,
 'output_chunk_id': 32069,
 'schema_conversion_id': 5,
 'schema_id': 28438}

In [51]:
print(dataset['train'][0]['input_data'])

                        1        2       3                                    4                                       5                                                       6                                                                                                                                                                                 7                                                  8       9                10  11   12                       13  14                                               15                   16                     17  18             19
      kelly10@example.net  710.460 -870865 e98d151e-9a67-4de2-bd87-ceca7be5f5b3 690a:3f0f:7f98:3602:f5cc:8994:ef71:2fbd                              As owner effect same make.                                                                                                   Where single age miss stop. Show heart garden fire economy career it operation.  9574 Dudley Courts Apt. 262\nGarciaview, TX 36247 -791056       

In [52]:
print(json.dumps(json.loads(dataset['train'][0]['schema']), indent=1))

{
 "type": [
  "array",
  "null"
 ],
 "items": {
  "type": [
   "object",
   "null"
  ],
  "properties": {
   "1": {
    "type": [
     "string",
     "null"
    ]
   },
   "2": {
    "type": [
     "number",
     "null"
    ]
   },
   "3": {
    "type": [
     "integer",
     "null"
    ]
   },
   "4": {
    "type": [
     "string",
     "null"
    ]
   },
   "5": {
    "type": [
     "string",
     "null"
    ]
   },
   "6": {
    "type": [
     "string",
     "null"
    ]
   },
   "7": {
    "type": [
     "string",
     "null"
    ]
   },
   "8": {
    "type": [
     "string",
     "null"
    ]
   },
   "9": {
    "type": [
     "integer",
     "null"
    ]
   },
   "10": {
    "type": [
     "string",
     "null"
    ]
   },
   "11": {
    "type": [
     "integer",
     "null"
    ]
   },
   "12": {
    "type": [
     "string",
     "null"
    ]
   },
   "13": {
    "type": [
     "string",
     "null"
    ]
   },
   "14": {
    "type": [
     "string",
     "null"
    ]
   },
   

In [53]:
print(json.dumps(json.loads(dataset['train'][0]['output']), indent=1))

[
 {
  "1": "kelly10@example.net",
  "2": 710.46,
  "3": -870865,
  "4": "e98d151e-9a67-4de2-bd87-ceca7be5f5b3",
  "5": "690a:3f0f:7f98:3602:f5cc:8994:ef71:2fbd",
  "6": "As owner effect same make.",
  "7": "Where single age miss stop. Show heart garden fire economy career it operation.",
  "8": "9574 Dudley Courts Apt. 262\nGarciaview, TX 36247",
  "9": -791056,
  "10": "Cuban peso",
  "11": -98,
  "12": null,
  "13": "https://hampton.org/",
  "14": "IMP",
  "15": "Loss entire campaign opportunity.",
  "16": "384-664-3967x67399",
  "17": "001-715-205-3942x78229",
  "18": "MVR",
  "19": "45.180.130.43"
 },
 {
  "1": "mark73@example.org",
  "2": -606.6,
  "3": 789856,
  "4": "f9a4ab1f-bacc-4b23-a527-fe18570bf0bf",
  "5": "8883:74a9:aca0:507b:351c:bf3d:d04b:4f97",
  "6": "Century defense commercial where week second remain by.",
  "7": "Say my size far Republican interest century. Government front after source.",
  "8": "558 Stephanie Trace\nLarryhaven, MI 29971",
  "9": -926706,
  "10":

# Run Benchmarking

In [54]:
DATASET_ID

'btseytlin/any2json'

In [ ]:
!python any2json/scripts/benchmark.py --hf-dataset btseytlin/any2json --split test --model-type qwen --output-dir=benchmark_results --model-kwargs='{"enable_thinking": true, "backend": "vllm_server", "server_log_path": "vllm.log"}'

2025-08-10 17:24:18.705133: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-10 17:24:18.722752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754846658.743716   21446 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754846658.750174   21446 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754846658.766452   21446 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 